In [1]:
import pandas as pd
from birdsong_dataset import AudioDataset , SoundData
from torch.utils.data import random_split, DataLoader
import numpy as np
import torch
import cv2 as cv
from matplotlib import pyplot as plt
from torch import nn, optim
import torch


In [16]:
df = pd.read_csv("./data/birdsong_metadata.csv")
df.nunique()
device = "cuda"

In [38]:

class transpose_layers(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        # print(sample.shape)
        
        transposed = sample.transpose((2, 0, 1))
        return transposed


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        # print(sample.shape)
        
        
        return torch.tensor(sample[0]).to(device) , torch.tensor(sample[1]).to(device) , torch.tensor(sample[2]).to(device)


In [ ]:
sounds = SoundData(df , transform = transpose_layers(), max_len= 4 , overlapping = 0 , noise_reduction=False, noise_reduction_level=0.6)
dataset = AudioDataset(sounds.train , transform = ToTensor())



Loading The Data...


In [6]:
# plt.figure(figsize=(10,10))
# counter = 1
# for i in range(1):
#     for j in range(1):
#         plt.subplot(1,1,counter)
#         counter+=1
#         plt.title(train[i][1].item())
#         if j % 2 == 0 :
#             plt.imshow(train[i][0].to('cpu').numpy().transpose((1,2,0)))
#         else:
#             plt.imshow(train[i][0].to('cpu').numpy().transpose((1,2,0)))
        
             

In [7]:
# from matplotlib import pyplot as plt
# plt.figure(figsize=(10,10))
# for i in range(8):
#     plt.subplot(2,4,i+1)
#     plt.title(train[i][1].item())
#     plt.imshow(train[i][0].to('cpu').numpy().transpose((1,2,0)))

In [42]:



class BirdSongClassifier(nn.Module):
    def __init__(self, num_classes_1 = 88 , num_classes_2 = 85):
        super(BirdSongClassifier,self).__init__()
        # backbone
        self.conv1 = nn.Conv2d( in_channels=1, out_channels= 64 , kernel_size=3)
        self.norm1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d((3,3))
        self.conv2 = nn.Conv2d( in_channels=64, out_channels=128 , kernel_size=3)
        self.norm2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d((3,3))
        self.conv3 = nn.Conv2d( in_channels=128, out_channels=64 , kernel_size=3)
        self.norm3 = nn.BatchNorm2d(64)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d((3,3))
        
        # multilabel classification for name 
        
        self.names_l1 = nn.Linear(64*11*2 ,512 )
        self.names_relu1 = nn.ReLU()
        self.names_l2 = nn.Linear(512 , num_classes_1 )
        
        # multilabel classification for species       
        
        # self.species_l1 = nn.Linear(64*11*2 ,512 )
        # self.species_relu1 = nn.ReLU()
        # self.species_l2 = nn.Linear(512 , num_classes_2 )
        # self.to(device)

        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.norm1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        backbone = self.norm3(x)
        # x = self.relu3(x)
        # backbone = self.pool3(x)
        # print("Backbone Shape : ", backbone.size())
        #forward for names
        x1 = backbone.view(-1, 64*11*2)
        x1 = self.names_l1(x1.reshape(-1,64*11*2))
        x1 = self.names_relu1(x1)
        x1 = self.names_l2(x1)
        # print("Name Head Shape = " ,x1.shape)

        #forward for species

        # x2 = backbone.view(-1, 64*11*2)
        # x2 = self.species_l1(x2.reshape(-1,64*11*2))
        # x2 = self.species_relu1(x2)
        # x2 = self.species_l2(x2)
        return x1 

    def test_model(self, data_loader):
        """
        Test a model on a specified dataset.
    
        Inputs:
            net - Trained model of type BaseNetwork
            data_loader - DataLoader object of the dataset to test on (validation or test)
        """
        self.eval()
        
        true_preds_names, true_preds_species,  count = 0., 0., 0
        
        for imgs, names , species in data_loader: 
            imgs, names , species = imgs.to(device), names.to(device) , species.to(device)
            with torch.no_grad():
                preds = self(imgs)
                preds_names = preds[0].argmax(dim=-1)
                # preds_species = preds[1].argmax(dim=-1)
                true_preds_names += (preds_names == names )[0].sum().item()
                # true_preds_species += (preds_species == species )[0].sum().item()
                count += 2
        test_acc = (true_preds_names +  true_preds_species) / count
        return test_acc
model = BirdSongClassifier()
model.to(device)


BirdSongClassifier(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (names_l1): Linear(in_features=1408, out_features=512, bias=True)
  (names_relu1): ReLU()
  (names_l2): Linear(in_features=512, out_features=88, bias=True)
)

In [46]:

# Define the sizes of your desired splits
train_size = int(0.6 * len(dataset))  # 60% of the data for training
val_size = int(0.25 * len(dataset))   # 25% of the data for validation
test_size = len(dataset) - train_size - val_size  # Remaining data for testing

# Use random_split with shuffled indices to create the splits
train, val, test = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

batch_size = 2
# You can then create data loaders for each split if needed
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

model.test_model(test_loader)

0.45151515151515154

In [47]:

import os



# Define loss function and optimizer
criterion_names = nn.CrossEntropyLoss()
criterion_species = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001 , weight_decay=0.01, momentum=0.9)
max_val_accuracy = 0 
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels_names, labels_species) in enumerate(train_loader):
        inputs , labels_names , _ = inputs , labels_names , labels_species
        optimizer.zero_grad()
        # Forward pass
        outputs_names  = model(inputs.type(torch.cuda.FloatTensor))
        # print(outputs_names.shape)
        # print(labels_names.shape)
        # print(torch.argmax(outputs_names , dim=-1).item(), labels_names.item() ,"-------", torch.argmax(outputs_species , dim=-1).item(), labels_species.item())
        # time.sleep(0.1)
        
        # Calculate loss
        loss_names = criterion_names(outputs_names, labels_names.type(torch.cuda.LongTensor))
        # loss_species = criterion_species(outputs_species, labels_species.type(torch.LongTensor).to(device))
        t_loss_names = loss_names 
        # t_loss_species = loss_species
        # print(t_loss_names , t_loss_species)
        total_loss = loss_names
        # Backward pass
        total_loss.backward()
        # t_loss_species.backward(retain_graph= True)
        # Update weights
        optimizer.step()
        # print(total_loss)
        # optimizer_species.step()
        # print(total_loss)
        running_loss += total_loss.item()
        # running_loss_species += t_loss_species.item()
        
        # Print statistics every 100 batches
    if (epoch + 1) % 1 == 0:
        train_acc = model.test_model(train_loader) 
        val_acc = model.test_model(val_loader)
        if epoch > 0 and val_acc > 0.6 and val_acc > max_val_accuracy :
            torch.save({
            'epoch': num_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, f"best.pt")
            max_val_accuracy = val_acc

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss : {batch_size * running_loss / len(train)  :.6f} , Train Acc : {train_acc:.4f}, Val Acc : {val_acc:.8f} , Best Acc : {max_val_accuracy:0.8f}')
    if (epoch + 1) % 10 == 0:
        test_acc = model.test_model(test_loader)
        print(f"TEST ACCURACY : {test_acc:0.4f}")
print('Finished Training')

Epoch [1/50], Training Loss : 0.254268 , Train Acc : 0.4751, Val Acc : 0.44611187 , Best Acc : 0.00000000
Epoch [2/50], Training Loss : 0.152521 , Train Acc : 0.4883, Val Acc : 0.45975443 , Best Acc : 0.00000000
Epoch [3/50], Training Loss : 0.118688 , Train Acc : 0.4779, Val Acc : 0.44884038 , Best Acc : 0.00000000
Epoch [4/50], Training Loss : 0.074432 , Train Acc : 0.4809, Val Acc : 0.45247840 , Best Acc : 0.00000000
Epoch [5/50], Training Loss : 0.053257 , Train Acc : 0.4899, Val Acc : 0.46066394 , Best Acc : 0.00000000
Epoch [6/50], Training Loss : 0.040441 , Train Acc : 0.4873, Val Acc : 0.46452933 , Best Acc : 0.00000000
Epoch [7/50], Training Loss : 0.028402 , Train Acc : 0.4989, Val Acc : 0.47407913 , Best Acc : 0.00000000
Epoch [8/50], Training Loss : 0.024509 , Train Acc : 0.4890, Val Acc : 0.46066394 , Best Acc : 0.00000000
Epoch [9/50], Training Loss : 0.020099 , Train Acc : 0.4867, Val Acc : 0.45861755 , Best Acc : 0.00000000
Epoch [10/50], Training Loss : 0.017519 , Trai

KeyboardInterrupt: 

In [12]:
# device = torch.device("cuda")
# model_ft = BirdSongClassifier()
# optimizer_ft = optim.SGD(model.parameters(), lr=0.00001)
# 
# checkpoint = torch.load("best.pt" , map_location= device)
# model_ft.load_state_dict(checkpoint['model_state_dict'])
# optimizer_ft.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# model_ft.to(device)

BirdSongClassifier(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (names_l1): Linear(in_features=1408, out_features=512, bias=True)
  (names_relu1): ReLU()
  (names_l2): Linear(in_features=512, out_features=88, bias=True)
  (species_l1): Linear(in_features=1408, out_features=512, bias=True)
  (species_relu1): ReLU()
  (species_l2): Linear(in_features=512, 

In [13]:
# num_epochs = 1000
# for epoch in range(num_epochs):
#     model_ft.train()
#     running_loss = 0.0
#     for i, (inputs, labels_names, labels_species) in enumerate(train_loader):
# 
#         labels_names , labels_species = labels_names.to("cuda") , labels_species.to("cuda")
#         optimizer.zero_grad()
#         # Forward pass
#         outputs_names , outputs_species = model_ft(inputs.type(torch.cuda.FloatTensor))
#         # print(torch.argmax(outputs_names , dim=-1).item(), labels_names.item() ,"-------", torch.argmax(outputs_species , dim=-1).item(), labels_species.item())
#         # time.sleep(0.1)
# 
#         # Calculate loss
#         loss_names = criterion_names(outputs_names, labels_names.type(torch.cuda.LongTensor) )
#         loss_species = criterion_species(outputs_species, labels_species.type(torch.cuda.LongTensor))
#         t_loss_names = loss_names 
#         t_loss_species = loss_species
#         total_loss = loss_names + loss_species
#         # Backward pass
#         total_loss.backward()
#         # t_loss_species.backward(retain_graph= True)
#         # Update weights
#         optimizer.step()
#         # optimizer_species.step()
#         running_loss += total_loss.item()
#         # running_loss_species += t_loss_species.item()
# 
#         # Print statistics every 100 batches
#     if (epoch + 1) % 1 == 0:
#         train_acc = model_ft.test_model(train_loader) 
#         val_acc = model_ft.test_model(val_loader)
# 
#         print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss : {running_loss :.6f} , Train Acc : {train_acc:.4f}, Val Acc : {val_acc:.8f}')


print('Finished Training')

KeyboardInterrupt: 

In [ ]:
input = torch.randn(1, 5, requires_grad=True)
input.shape

In [ ]:
target = torch.empty(1, dtype=torch.long).random_(5)
target.shape

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

model = resnet50(pretrained=True)
target_layers = [model.layer4[-1]]
input_tensor = # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.

targets = [ClassifierOutputTarget(281)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

# You can also get the model outputs without having to re-inference
model_outputs = cam.outputs